In [1]:
import librosa
import librosa.display
import os
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random

/usr/local/lib/python3.6/dist-packages/numba/errors.py:137: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)typ

In [2]:
#currdir= os.getcwd()
rootdir = '/home/taejoon/PhysioNetChallenge'
input_directory = os.path.join(rootdir, 'Training_WFDB')
mel_directory = os.path.join(rootdir, 'Mel_data')
#save_directory = os.path.join(currdir, '')
if not os.path.isdir(input_directory):
        os.mkdir(input_directory)
if not os.path.isdir(mel_directory):
        os.mkdir(mel_directory)        

In [3]:
# Find files
input_files = []
for f in os.listdir(input_directory):
    if os.path.isfile(os.path.join(input_directory, f)) and not f.lower().startswith('.') and f.lower().endswith('mat'):
        input_files.append(f)

In [4]:
input_file_names = sorted(input_files)
input_file_names

['A0001.mat',
 'A0002.mat',
 'A0003.mat',
 'A0004.mat',
 'A0005.mat',
 'A0006.mat',
 'A0007.mat',
 'A0008.mat',
 'A0009.mat',
 'A0010.mat',
 'A0011.mat',
 'A0012.mat',
 'A0013.mat',
 'A0014.mat',
 'A0015.mat',
 'A0016.mat',
 'A0017.mat',
 'A0018.mat',
 'A0019.mat',
 'A0020.mat',
 'A0021.mat',
 'A0022.mat',
 'A0023.mat',
 'A0024.mat',
 'A0025.mat',
 'A0026.mat',
 'A0027.mat',
 'A0028.mat',
 'A0029.mat',
 'A0030.mat',
 'A0031.mat',
 'A0032.mat',
 'A0033.mat',
 'A0034.mat',
 'A0035.mat',
 'A0036.mat',
 'A0037.mat',
 'A0038.mat',
 'A0039.mat',
 'A0040.mat',
 'A0041.mat',
 'A0042.mat',
 'A0043.mat',
 'A0044.mat',
 'A0045.mat',
 'A0046.mat',
 'A0047.mat',
 'A0048.mat',
 'A0049.mat',
 'A0050.mat',
 'A0051.mat',
 'A0052.mat',
 'A0053.mat',
 'A0054.mat',
 'A0055.mat',
 'A0056.mat',
 'A0057.mat',
 'A0058.mat',
 'A0059.mat',
 'A0060.mat',
 'A0061.mat',
 'A0062.mat',
 'A0063.mat',
 'A0064.mat',
 'A0065.mat',
 'A0066.mat',
 'A0067.mat',
 'A0068.mat',
 'A0069.mat',
 'A0070.mat',
 'A0071.mat',
 'A007

In [5]:
np.shape(input_file_names)

(6877,)

### Get Y data

In [6]:
# Find unique number of classes  
def get_unique_classes(input_directory,files):

    unique_classes=set()
    for f in files:
        g = f.replace('.mat','.hea')
        input_file = os.path.join(input_directory,g)
        with open(input_file,'r') as f:
            for lines in f:
                if lines.startswith('#Dx'):
                    tmp = lines.split(': ')[1].split(',')
                    for c in tmp:
                        unique_classes.add(c.strip())

    return sorted(unique_classes)

unique_classes = get_unique_classes(input_directory, input_files)
# Creating one-hot vector for Y
# num = np.unique(classes, axis=0)
class2index = {}
for a, b in enumerate(unique_classes):
    class2index[b] = a
#class2index

def one_hot_encoding(y, class2index):
       one_hot_vector = [0]*(len(class2index))
       ind=class2index[y]
       one_hot_vector[ind]=1
       return one_hot_vector


In [7]:
class2index

{'AF': 0,
 'I-AVB': 1,
 'LBBB': 2,
 'Normal': 3,
 'PAC': 4,
 'PVC': 5,
 'RBBB': 6,
 'STD': 7,
 'STE': 8}

In [8]:

# Get classes of sorted file names
def get_classes(input_directory,files):

#     classes=set()
    classes = []
    for f in files:
        g = f.replace('.mat','.hea')
        input_file = os.path.join(input_directory,g)
        with open(input_file,'r') as f:
            for lines in f:
                if lines.startswith('#Dx'):
                    tmp = lines.split(': ')[1].split(',')
                    for c in tmp:
#                         curr_label = c.strip()
                        curr_label = one_hot_encoding(c.strip(), class2index)
                    classes.append(curr_label)

    return classes

classes = get_classes(input_directory, input_file_names)
classes

[[0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0,

In [9]:
np.shape(classes)

(6877, 9)

### Now, get X data

In [10]:
input_file_names

['A0001.mat',
 'A0002.mat',
 'A0003.mat',
 'A0004.mat',
 'A0005.mat',
 'A0006.mat',
 'A0007.mat',
 'A0008.mat',
 'A0009.mat',
 'A0010.mat',
 'A0011.mat',
 'A0012.mat',
 'A0013.mat',
 'A0014.mat',
 'A0015.mat',
 'A0016.mat',
 'A0017.mat',
 'A0018.mat',
 'A0019.mat',
 'A0020.mat',
 'A0021.mat',
 'A0022.mat',
 'A0023.mat',
 'A0024.mat',
 'A0025.mat',
 'A0026.mat',
 'A0027.mat',
 'A0028.mat',
 'A0029.mat',
 'A0030.mat',
 'A0031.mat',
 'A0032.mat',
 'A0033.mat',
 'A0034.mat',
 'A0035.mat',
 'A0036.mat',
 'A0037.mat',
 'A0038.mat',
 'A0039.mat',
 'A0040.mat',
 'A0041.mat',
 'A0042.mat',
 'A0043.mat',
 'A0044.mat',
 'A0045.mat',
 'A0046.mat',
 'A0047.mat',
 'A0048.mat',
 'A0049.mat',
 'A0050.mat',
 'A0051.mat',
 'A0052.mat',
 'A0053.mat',
 'A0054.mat',
 'A0055.mat',
 'A0056.mat',
 'A0057.mat',
 'A0058.mat',
 'A0059.mat',
 'A0060.mat',
 'A0061.mat',
 'A0062.mat',
 'A0063.mat',
 'A0064.mat',
 'A0065.mat',
 'A0066.mat',
 'A0067.mat',
 'A0068.mat',
 'A0069.mat',
 'A0070.mat',
 'A0071.mat',
 'A007

In [11]:
mel_directory

'/home/taejoon/PhysioNetChallenge/Mel_data'

In [12]:
# Checkinc which x is minimum
minimum = 20
for file in input_file_names:
    tmp_file = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
    print(np.shape(tmp_file))
    if len(tmp_file) < minimum:
        minimum = tmp_file.shape[0]
print(minimum)

(26, 224, 12)
(18, 224, 12)
(18, 224, 12)
(21, 224, 12)
(43, 224, 12)
(24, 224, 12)
(18, 224, 12)
(26, 224, 12)
(28, 224, 12)
(18, 224, 12)
(62, 224, 12)
(29, 224, 12)
(29, 224, 12)
(27, 224, 12)
(43, 224, 12)
(35, 224, 12)
(77, 224, 12)
(86, 224, 12)
(19, 224, 12)
(29, 224, 12)
(18, 224, 12)
(46, 224, 12)
(18, 224, 12)
(40, 224, 12)
(18, 224, 12)
(19, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(21, 224, 12)
(38, 224, 12)
(35, 224, 12)
(18, 224, 12)
(18, 224, 12)
(19, 224, 12)
(54, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(53, 224, 12)
(18, 224, 12)
(21, 224, 12)
(26, 224, 12)
(18, 224, 12)
(18, 224, 12)
(61, 224, 12)
(33, 224, 12)
(20, 224, 12)
(18, 224, 12)
(24, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(19, 224, 12)
(80, 224, 12)
(19, 224, 12)
(29, 224, 12)
(79, 224, 12)
(35, 224, 12)
(20, 224, 12)
(18, 224, 12)
(52, 224, 12)
(22, 224, 12)
(18, 224, 12)
(19, 224, 12)
(18, 224, 12)
(31, 224, 12)
(18, 224, 12)
(71, 224, 12)
(21, 224, 12)
(29, 224, 12)
(28, 2

(41, 224, 12)
(24, 224, 12)
(28, 224, 12)
(18, 224, 12)
(35, 224, 12)
(19, 224, 12)
(102, 224, 12)
(41, 224, 12)
(23, 224, 12)
(18, 224, 12)
(23, 224, 12)
(18, 224, 12)
(23, 224, 12)
(27, 224, 12)
(18, 224, 12)
(19, 224, 12)
(18, 224, 12)
(57, 224, 12)
(18, 224, 12)
(19, 224, 12)
(18, 224, 12)
(20, 224, 12)
(26, 224, 12)
(18, 224, 12)
(33, 224, 12)
(19, 224, 12)
(59, 224, 12)
(21, 224, 12)
(24, 224, 12)
(18, 224, 12)
(30, 224, 12)
(48, 224, 12)
(67, 224, 12)
(18, 224, 12)
(21, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(63, 224, 12)
(22, 224, 12)
(18, 224, 12)
(26, 224, 12)
(23, 224, 12)
(84, 224, 12)
(25, 224, 12)
(28, 224, 12)
(24, 224, 12)
(24, 224, 12)
(58, 224, 12)
(103, 224, 12)
(35, 224, 12)
(46, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(19, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(26, 224, 12)
(24, 224, 12)
(90, 224, 12)
(29, 224, 12)
(35, 224, 12)
(18, 224, 12)
(20, 224, 12)
(54, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18,

(24, 224, 12)
(18, 224, 12)
(58, 224, 12)
(26, 224, 12)
(20, 224, 12)
(18, 224, 12)
(31, 224, 12)
(21, 224, 12)
(76, 224, 12)
(18, 224, 12)
(18, 224, 12)
(38, 224, 12)
(35, 224, 12)
(20, 224, 12)
(18, 224, 12)
(18, 224, 12)
(24, 224, 12)
(35, 224, 12)
(35, 224, 12)
(18, 224, 12)
(33, 224, 12)
(18, 224, 12)
(28, 224, 12)
(18, 224, 12)
(75, 224, 12)
(49, 224, 12)
(28, 224, 12)
(40, 224, 12)
(24, 224, 12)
(18, 224, 12)
(40, 224, 12)
(18, 224, 12)
(26, 224, 12)
(21, 224, 12)
(36, 224, 12)
(18, 224, 12)
(19, 224, 12)
(26, 224, 12)
(18, 224, 12)
(54, 224, 12)
(19, 224, 12)
(63, 224, 12)
(18, 224, 12)
(21, 224, 12)
(18, 224, 12)
(19, 224, 12)
(50, 224, 12)
(18, 224, 12)
(24, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(22, 224, 12)
(22, 224, 12)
(19, 224, 12)
(36, 224, 12)
(29, 224, 12)
(33, 224, 12)
(28, 224, 12)
(28, 224, 12)
(29, 224, 12)
(23, 224, 12)
(19, 224, 12)
(18, 224, 12)
(18, 224, 12)
(19, 224, 12)
(20, 224, 12)
(20, 224, 12)
(18, 224, 12)
(21, 224, 12)
(52, 2

(21, 224, 12)
(63, 224, 12)
(23, 224, 12)
(18, 224, 12)
(19, 224, 12)
(38, 224, 12)
(21, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(20, 224, 12)
(18, 224, 12)
(23, 224, 12)
(25, 224, 12)
(18, 224, 12)
(28, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(36, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(36, 224, 12)
(19, 224, 12)
(18, 224, 12)
(26, 224, 12)
(18, 224, 12)
(18, 224, 12)
(19, 224, 12)
(24, 224, 12)
(32, 224, 12)
(29, 224, 12)
(23, 224, 12)
(35, 224, 12)
(31, 224, 12)
(18, 224, 12)
(26, 224, 12)
(38, 224, 12)
(19, 224, 12)
(18, 224, 12)
(21, 224, 12)
(46, 224, 12)
(24, 224, 12)
(58, 224, 12)
(24, 224, 12)
(18, 224, 12)
(36, 224, 12)
(38, 224, 12)
(18, 224, 12)
(18, 224, 12)
(22, 224, 12)
(18, 224, 12)
(26, 224, 12)
(18, 224, 12)
(27, 224, 12)
(36, 224, 12)
(18, 224, 12)
(18, 224, 12)
(19, 224, 12)
(18, 224, 12)
(46, 224, 12)
(35, 224, 12)
(19, 224, 12)
(18, 224, 12)
(18, 224, 12)
(78, 224, 12)
(23, 2

(33, 224, 12)
(53, 224, 12)
(55, 224, 12)
(45, 224, 12)
(18, 224, 12)
(52, 224, 12)
(21, 224, 12)
(29, 224, 12)
(24, 224, 12)
(19, 224, 12)
(23, 224, 12)
(18, 224, 12)
(35, 224, 12)
(28, 224, 12)
(19, 224, 12)
(32, 224, 12)
(21, 224, 12)
(48, 224, 12)
(19, 224, 12)
(40, 224, 12)
(29, 224, 12)
(24, 224, 12)
(23, 224, 12)
(25, 224, 12)
(36, 224, 12)
(18, 224, 12)
(22, 224, 12)
(19, 224, 12)
(19, 224, 12)
(21, 224, 12)
(18, 224, 12)
(53, 224, 12)
(33, 224, 12)
(24, 224, 12)
(18, 224, 12)
(19, 224, 12)
(24, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(26, 224, 12)
(53, 224, 12)
(35, 224, 12)
(26, 224, 12)
(52, 224, 12)
(35, 224, 12)
(22, 224, 12)
(21, 224, 12)
(19, 224, 12)
(20, 224, 12)
(18, 224, 12)
(38, 224, 12)
(18, 224, 12)
(40, 224, 12)
(29, 224, 12)
(18, 224, 12)
(21, 224, 12)
(18, 224, 12)
(20, 224, 12)
(38, 224, 12)
(52, 224, 12)
(18, 224, 12)
(28, 224, 12)
(29, 224, 12)
(22, 224, 12)
(19, 224, 12)
(29, 224, 12)
(21, 224, 12)
(18, 224, 12)
(19, 224, 12)
(26, 224, 12)
(18, 2

(74, 224, 12)
(24, 224, 12)
(24, 224, 12)
(22, 224, 12)
(19, 224, 12)
(20, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(35, 224, 12)
(50, 224, 12)
(56, 224, 12)
(18, 224, 12)
(20, 224, 12)
(18, 224, 12)
(19, 224, 12)
(36, 224, 12)
(18, 224, 12)
(19, 224, 12)
(57, 224, 12)
(28, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(31, 224, 12)
(26, 224, 12)
(20, 224, 12)
(28, 224, 12)
(18, 224, 12)
(18, 224, 12)
(31, 224, 12)
(35, 224, 12)
(35, 224, 12)
(43, 224, 12)
(18, 224, 12)
(38, 224, 12)
(26, 224, 12)
(41, 224, 12)
(57, 224, 12)
(18, 224, 12)
(22, 224, 12)
(35, 224, 12)
(20, 224, 12)
(103, 224, 12)
(58, 224, 12)
(21, 224, 12)
(18, 224, 12)
(18, 224, 12)
(28, 224, 12)
(40, 224, 12)
(18, 224, 12)
(23, 224, 12)
(84, 224, 12)
(36, 224, 12)
(26, 224, 12)
(25, 224, 12)
(26, 224, 12)
(18, 224, 12)
(46, 224, 12)
(33, 224, 12)
(18, 224, 12)
(21, 224, 12)
(18, 224, 12)
(18, 224, 12)
(35, 224, 12)
(18, 224, 12)
(26, 224, 12)
(18, 224, 12)
(27, 224, 12)
(20, 224, 12)
(18, 

(38, 224, 12)
(22, 224, 12)
(38, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(67, 224, 12)
(23, 224, 12)
(18, 224, 12)
(20, 224, 12)
(18, 224, 12)
(18, 224, 12)
(21, 224, 12)
(49, 224, 12)
(18, 224, 12)
(23, 224, 12)
(27, 224, 12)
(41, 224, 12)
(80, 224, 12)
(18, 224, 12)
(24, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(103, 224, 12)
(26, 224, 12)
(21, 224, 12)
(47, 224, 12)
(18, 224, 12)
(18, 224, 12)
(46, 224, 12)
(19, 224, 12)
(68, 224, 12)
(18, 224, 12)
(18, 224, 12)
(74, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(38, 224, 12)
(48, 224, 12)
(51, 224, 12)
(18, 224, 12)
(20, 224, 12)
(18, 224, 12)
(18, 224, 12)
(19, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(26, 224, 12)
(21, 224, 12)
(20, 224, 12)
(20, 224, 12)
(79, 224, 12)
(18, 224, 12)
(18, 224, 12)
(30, 224, 12)
(91, 224, 12)
(18, 224, 12)
(19, 224, 12)
(24, 224, 12)
(36, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 

(48, 224, 12)
(21, 224, 12)
(18, 224, 12)
(33, 224, 12)
(18, 224, 12)
(20, 224, 12)
(29, 224, 12)
(18, 224, 12)
(60, 224, 12)
(19, 224, 12)
(18, 224, 12)
(18, 224, 12)
(73, 224, 12)
(18, 224, 12)
(18, 224, 12)
(21, 224, 12)
(18, 224, 12)
(19, 224, 12)
(19, 224, 12)
(21, 224, 12)
(58, 224, 12)
(18, 224, 12)
(18, 224, 12)
(21, 224, 12)
(18, 224, 12)
(20, 224, 12)
(31, 224, 12)
(18, 224, 12)
(26, 224, 12)
(18, 224, 12)
(18, 224, 12)
(34, 224, 12)
(35, 224, 12)
(20, 224, 12)
(20, 224, 12)
(26, 224, 12)
(18, 224, 12)
(18, 224, 12)
(37, 224, 12)
(18, 224, 12)
(18, 224, 12)
(19, 224, 12)
(25, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(38, 224, 12)
(18, 224, 12)
(18, 224, 12)
(28, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(72, 224, 12)
(18, 224, 12)
(57, 224, 12)
(18, 224, 12)
(18, 224, 12)
(71, 224, 12)
(18, 224, 12)
(33, 224, 12)
(31, 224, 12)
(18, 224, 12)
(21, 224, 12)
(22, 224, 12)
(18, 224, 12)
(21, 224, 12)
(42, 224, 12)
(19, 224, 12)
(19, 224, 12)
(21, 224, 12)
(46, 2

(33, 224, 12)
(57, 224, 12)
(18, 224, 12)
(18, 224, 12)
(35, 224, 12)
(25, 224, 12)
(28, 224, 12)
(27, 224, 12)
(23, 224, 12)
(20, 224, 12)
(26, 224, 12)
(18, 224, 12)
(18, 224, 12)
(29, 224, 12)
(103, 224, 12)
(18, 224, 12)
(26, 224, 12)
(20, 224, 12)
(18, 224, 12)
(35, 224, 12)
(21, 224, 12)
(26, 224, 12)
(26, 224, 12)
(18, 224, 12)
(21, 224, 12)
(18, 224, 12)
(18, 224, 12)
(28, 224, 12)
(18, 224, 12)
(21, 224, 12)
(35, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(26, 224, 12)
(21, 224, 12)
(26, 224, 12)
(41, 224, 12)
(36, 224, 12)
(18, 224, 12)
(21, 224, 12)
(70, 224, 12)
(79, 224, 12)
(35, 224, 12)
(18, 224, 12)
(18, 224, 12)
(19, 224, 12)
(24, 224, 12)
(18, 224, 12)
(21, 224, 12)
(31, 224, 12)
(35, 224, 12)
(18, 224, 12)
(21, 224, 12)
(19, 224, 12)
(55, 224, 12)
(26, 224, 12)
(19, 224, 12)
(36, 224, 12)
(33, 224, 12)
(18, 224, 12)
(60, 224, 12)
(22, 224, 12)
(19, 224, 12)
(50, 224, 12)
(28, 224, 12)
(25, 224, 12)
(25, 224, 12)
(21, 224, 12)
(18, 224, 12)
(35, 

(50, 224, 12)
(18, 224, 12)
(18, 224, 12)
(24, 224, 12)
(31, 224, 12)
(18, 224, 12)
(18, 224, 12)
(20, 224, 12)
(18, 224, 12)
(21, 224, 12)
(25, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(19, 224, 12)
(24, 224, 12)
(61, 224, 12)
(68, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(18, 224, 12)
(21, 224, 12)
(29, 224, 12)
(19, 224, 12)
(77, 224, 12)
(18, 224, 12)
(26, 224, 12)
(18, 224, 12)
(67, 224, 12)
(24, 224, 12)
(18, 224, 12)
(18, 224, 12)
(23, 224, 12)
(20, 224, 12)
(28, 224, 12)
(38, 224, 12)
(29, 224, 12)
(23, 224, 12)
(18, 224, 12)
(23, 224, 12)
(28, 224, 12)
(22, 224, 12)
(23, 224, 12)
(21, 224, 12)
(25, 224, 12)
(33, 224, 12)
(23, 224, 12)
(23, 224, 12)
(29, 224, 12)
(21, 224, 12)
(21, 224, 12)
(18, 224, 12)
(36, 224, 12)
(20, 224, 12)
(18, 224, 12)
(20, 224, 12)
(19, 224, 12)
(19, 224, 12)
(18, 224, 12)
(33, 224, 12)
(29, 224, 12)
(22, 224, 12)
(23, 224, 12)
(61, 224, 12)
(18, 224, 12)
(18, 224, 12)
(35, 224, 12)
(22, 224, 12)
(35, 224, 12)
(18, 224, 12)
(18, 2

In [13]:
minimum_len = minimum
def block_feature(sequence_en): 
    new_en = []
    if len(sequence_en) > minimum_len:  # 길이가 minimum보다 긴 경우
        start = random.randint(0,len(sequence_en)-minimum_len)    
        new_en = sequence_en[start:start+minimum_len]
    elif len(sequence_en) == minimum_len: # 길이가 minimum
        new_en = sequence_en
    else: 
        assert len(sequence_en) <= minimum_len
    return new_en

In [14]:
mel_files = []
for file in input_file_names:
    tmp_file = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
    clip_file = block_feature(tmp_file)
    mel_files.append(clip_file)

In [15]:
for i in range(len(mel_files)):
    print(np.shape(mel_files[i]))

(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 2

(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 224, 12)
(11, 2

-> 랜덤으로 minimum 길이 (11) 만큼의 x가 뽑힌 것 확인함

In [16]:
np.shape(mel_files)

(6877, 11, 224, 12)

In [60]:
#dataset = tf.data.Dataset.from_tensor_slices(mel_files).batch(10)
#dataset

In [18]:
# batch_size = 32
# batch_per_epoch = batch_size // len(mel_files)
# batch_per_epoch

### x, y 데이터 가공

In [19]:
classes= np.asarray(classes)    

In [21]:
mel_files = np.asarray(mel_files)

In [22]:
dataset = tf.data.Dataset.from_tensor_slices((mel_files, classes))

In [23]:
dataset

<DatasetV1Adapter shapes: ((11, 224, 12), (9,)), types: (tf.float32, tf.int64)>

In [25]:
batch_size = 32
dataset = dataset.batch(batch_size)
dataset

<DatasetV1Adapter shapes: ((?, 11, 224, 12), (?, 9)), types: (tf.float32, tf.int64)>

In [59]:
#iterator_mel = dataset.make_initializable_iterator()
# image_stacked, label_stacked = iterator.get_next()
# with tf.Session() as sess:
#     sess.run(iterator.initializer)
#     image, label = sess.run([image_stacked, label_stacked])
# image.shape
# label.shape

In [64]:
from sklearn.model_selection import train_test_split

In [65]:
# Set TF random seed to improve reproducibility
tf.set_random_seed(1234)

In [71]:
x, x_test, y, y_test  = train_test_split(mel_files, classes, test_size=0.2, train_size = 0.8)
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 0.25, train_size = 0.75)

In [72]:
print(np.shape(x_train), np.shape(x_val), np.shape(x_test), np.shape(y_train), np.shape(y_val), np.shape(y_test))

(4125, 11, 224, 12) (1376, 11, 224, 12) (1376, 11, 224, 12) (4125, 9) (1376, 9) (1376, 9)


### CNN Model

In [77]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(11, 224, 12)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
#model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
#model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(9, activation='softmax'))

In [80]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 9, 222, 32)        3488      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 111, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 2, 109, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 54, 64)         0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 3456)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               1769984   
_________________________________________________________________
dense_8 (Dense)              (None, 9)                 4617      
Total para

In [81]:
from keras import optimizers
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [82]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [73]:
# history = model.fit(
#       mel_files, classes)

In [74]:
# model.fit_generator(datagen.flow(mel_files, classes, batch_size=32), steps_per_epoch = len(mel_files) / 32, epochs = 20)

In [83]:
nepochs=1000
model.fit(x_train, y_train, batch_size=batch_size, epochs=nepochs, validation_data=(x_val, y_val), verbose=2)

Train on 4125 samples, validate on 1376 samples
Epoch 1/1000
 - 1s - loss: 2.1099 - acc: 0.2252 - val_loss: 2.0366 - val_acc: 0.2558
Epoch 2/1000
 - 1s - loss: 2.0104 - acc: 0.2669 - val_loss: 1.9732 - val_acc: 0.3198
Epoch 3/1000
 - 1s - loss: 1.9505 - acc: 0.2962 - val_loss: 1.9327 - val_acc: 0.3249
Epoch 4/1000
 - 1s - loss: 1.8944 - acc: 0.3236 - val_loss: 2.0059 - val_acc: 0.2994
Epoch 5/1000
 - 1s - loss: 1.8476 - acc: 0.3408 - val_loss: 1.8455 - val_acc: 0.3525
Epoch 6/1000
 - 1s - loss: 1.8028 - acc: 0.3537 - val_loss: 1.8508 - val_acc: 0.3234
Epoch 7/1000
 - 1s - loss: 1.7578 - acc: 0.3872 - val_loss: 1.8083 - val_acc: 0.3750
Epoch 8/1000
 - 1s - loss: 1.7152 - acc: 0.4039 - val_loss: 1.7831 - val_acc: 0.3903
Epoch 9/1000
 - 1s - loss: 1.6771 - acc: 0.4121 - val_loss: 1.9100 - val_acc: 0.3394
Epoch 10/1000
 - 1s - loss: 1.6450 - acc: 0.4291 - val_loss: 1.7224 - val_acc: 0.3983
Epoch 11/1000
 - 1s - loss: 1.6088 - acc: 0.4512 - val_loss: 1.7878 - val_acc: 0.3670
Epoch 12/1000
 

Epoch 96/1000
 - 1s - loss: 0.0603 - acc: 0.9913 - val_loss: 2.1665 - val_acc: 0.4855
Epoch 97/1000
 - 1s - loss: 0.0524 - acc: 0.9952 - val_loss: 2.1943 - val_acc: 0.4724
Epoch 98/1000
 - 1s - loss: 0.0485 - acc: 0.9944 - val_loss: 2.2263 - val_acc: 0.4833
Epoch 99/1000
 - 1s - loss: 0.0463 - acc: 0.9956 - val_loss: 2.2465 - val_acc: 0.4797
Epoch 100/1000
 - 1s - loss: 0.0491 - acc: 0.9930 - val_loss: 2.3656 - val_acc: 0.4826
Epoch 101/1000
 - 1s - loss: 0.0427 - acc: 0.9935 - val_loss: 2.3177 - val_acc: 0.4862
Epoch 102/1000
 - 1s - loss: 0.0397 - acc: 0.9944 - val_loss: 2.2874 - val_acc: 0.4847
Epoch 103/1000
 - 1s - loss: 0.0363 - acc: 0.9959 - val_loss: 2.3063 - val_acc: 0.4797
Epoch 104/1000
 - 1s - loss: 0.0379 - acc: 0.9942 - val_loss: 2.3252 - val_acc: 0.4753
Epoch 105/1000
 - 1s - loss: 0.0362 - acc: 0.9939 - val_loss: 2.3155 - val_acc: 0.4775
Epoch 106/1000
 - 1s - loss: 0.0405 - acc: 0.9905 - val_loss: 2.8560 - val_acc: 0.4353
Epoch 107/1000
 - 1s - loss: 0.0383 - acc: 0.99

Epoch 191/1000
 - 1s - loss: 0.0092 - acc: 0.9973 - val_loss: 3.4970 - val_acc: 0.4651
Epoch 192/1000
 - 1s - loss: 0.0054 - acc: 0.9990 - val_loss: 3.4545 - val_acc: 0.4840
Epoch 193/1000
 - 1s - loss: 0.0066 - acc: 0.9976 - val_loss: 3.5104 - val_acc: 0.4804
Epoch 194/1000
 - 1s - loss: 0.0153 - acc: 0.9956 - val_loss: 3.5295 - val_acc: 0.4760
Epoch 195/1000
 - 1s - loss: 0.0100 - acc: 0.9976 - val_loss: 3.5230 - val_acc: 0.4731
Epoch 196/1000
 - 1s - loss: 0.0066 - acc: 0.9990 - val_loss: 3.4993 - val_acc: 0.4826
Epoch 197/1000
 - 1s - loss: 0.0081 - acc: 0.9981 - val_loss: 4.5889 - val_acc: 0.4288
Epoch 198/1000
 - 1s - loss: 0.0075 - acc: 0.9981 - val_loss: 3.5332 - val_acc: 0.4717
Epoch 199/1000
 - 1s - loss: 0.0017 - acc: 0.9998 - val_loss: 3.5406 - val_acc: 0.4869
Epoch 200/1000
 - 1s - loss: 0.0142 - acc: 0.9947 - val_loss: 3.4791 - val_acc: 0.4702
Epoch 201/1000
 - 1s - loss: 0.0017 - acc: 0.9998 - val_loss: 3.5684 - val_acc: 0.4782
Epoch 202/1000
 - 1s - loss: 0.0043 - acc: 

 - 1s - loss: 0.0039 - acc: 0.9985 - val_loss: 4.1061 - val_acc: 0.4680
Epoch 286/1000
 - 1s - loss: 9.8683e-04 - acc: 0.9998 - val_loss: 4.0979 - val_acc: 0.4731
Epoch 287/1000
 - 1s - loss: 0.0062 - acc: 0.9985 - val_loss: 4.0468 - val_acc: 0.4818
Epoch 288/1000
 - 1s - loss: 0.0180 - acc: 0.9956 - val_loss: 3.9888 - val_acc: 0.4680
Epoch 289/1000
 - 1s - loss: 0.0053 - acc: 0.9988 - val_loss: 4.0910 - val_acc: 0.4826
Epoch 290/1000
 - 1s - loss: 0.0108 - acc: 0.9981 - val_loss: 4.1232 - val_acc: 0.4673
Epoch 291/1000
 - 1s - loss: 0.0049 - acc: 0.9981 - val_loss: 4.2254 - val_acc: 0.4702
Epoch 292/1000
 - 1s - loss: 0.0063 - acc: 0.9978 - val_loss: 4.0549 - val_acc: 0.4818
Epoch 293/1000
 - 1s - loss: 0.0032 - acc: 0.9985 - val_loss: 4.0990 - val_acc: 0.4826
Epoch 294/1000
 - 1s - loss: 0.0053 - acc: 0.9983 - val_loss: 4.0702 - val_acc: 0.4862
Epoch 295/1000
 - 1s - loss: 0.0056 - acc: 0.9983 - val_loss: 4.0650 - val_acc: 0.4760
Epoch 296/1000
 - 1s - loss: 0.0055 - acc: 0.9985 - va

Epoch 379/1000
 - 1s - loss: 0.0039 - acc: 0.9985 - val_loss: 4.6095 - val_acc: 0.4833
Epoch 380/1000
 - 1s - loss: 3.4015e-05 - acc: 1.0000 - val_loss: 4.6275 - val_acc: 0.4826
Epoch 381/1000
 - 1s - loss: 0.0041 - acc: 0.9988 - val_loss: 4.5553 - val_acc: 0.4746
Epoch 382/1000
 - 1s - loss: 0.0034 - acc: 0.9990 - val_loss: 4.4791 - val_acc: 0.4782
Epoch 383/1000
 - 1s - loss: 0.0075 - acc: 0.9983 - val_loss: 4.5444 - val_acc: 0.4767
Epoch 384/1000
 - 1s - loss: 8.9934e-06 - acc: 1.0000 - val_loss: 4.7361 - val_acc: 0.4855
Epoch 385/1000
 - 1s - loss: 0.0045 - acc: 0.9993 - val_loss: 4.5568 - val_acc: 0.4709
Epoch 386/1000
 - 1s - loss: 0.0049 - acc: 0.9976 - val_loss: 4.4346 - val_acc: 0.4898
Epoch 387/1000
 - 1s - loss: 0.0018 - acc: 0.9993 - val_loss: 4.4545 - val_acc: 0.4884
Epoch 388/1000
 - 1s - loss: 8.6789e-04 - acc: 0.9995 - val_loss: 4.4613 - val_acc: 0.4651
Epoch 389/1000
 - 1s - loss: 1.7931e-05 - acc: 1.0000 - val_loss: 4.6078 - val_acc: 0.4833
Epoch 390/1000
 - 1s - loss

...

Epoch 565/1000
 - 1s - loss: 1.7446e-05 - acc: 1.0000 - val_loss: 5.2369 - val_acc: 0.4717
Epoch 566/1000
 - 1s - loss: 0.0085 - acc: 0.9978 - val_loss: 5.1273 - val_acc: 0.4709
Epoch 567/1000
 - 1s - loss: 0.0093 - acc: 0.9988 - val_loss: 4.9529 - val_acc: 0.4644
Epoch 568/1000
 - 1s - loss: 0.0038 - acc: 0.9990 - val_loss: 5.0522 - val_acc: 0.4767
Epoch 569/1000
 - 1s - loss: 0.0027 - acc: 0.9990 - val_loss: 5.4886 - val_acc: 0.4651
Epoch 570/1000
 - 1s - loss: 1.1479e-04 - acc: 1.0000 - val_loss: 5.1981 - val_acc: 0.4680
Epoch 571/1000
 - 1s - loss: 0.0085 - acc: 0.9981 - val_loss: 5.2995 - val_acc: 0.4477
Epoch 572/1000
 - 1s - loss: 2.4566e-05 - acc: 1.0000 - val_loss: 5.2367 - val_acc: 0.4608
Epoch 573/1000
 - 1s - loss: 0.0047 - acc: 0.9985 - val_loss: 5.0995 - val_acc: 0.4782
Epoch 574/1000
 - 1s - loss: 7.3594e-06 - acc: 1.0000 - val_loss: 5.1359 - val_acc: 0.4709
Epoch 575/1000
 - 1s - loss: 0.0033 - acc: 0.9985 - val_loss: 5.0160 - val_acc: 0.4760
Epoch 576/1000
 - 1s - loss

Epoch 658/1000
 - 1s - loss: 0.0049 - acc: 0.9993 - val_loss: 5.2564 - val_acc: 0.4644
Epoch 659/1000
 - 1s - loss: 0.0051 - acc: 0.9981 - val_loss: 5.2385 - val_acc: 0.4746
Epoch 660/1000
 - 1s - loss: 6.8460e-04 - acc: 0.9995 - val_loss: 5.3367 - val_acc: 0.4622
Epoch 661/1000
 - 1s - loss: 0.0031 - acc: 0.9990 - val_loss: 5.2547 - val_acc: 0.4557
Epoch 662/1000
 - 1s - loss: 1.8927e-06 - acc: 1.0000 - val_loss: 5.3019 - val_acc: 0.4702
Epoch 663/1000
 - 1s - loss: 0.0047 - acc: 0.9993 - val_loss: 5.2480 - val_acc: 0.4695
Epoch 664/1000
 - 1s - loss: 0.0071 - acc: 0.9981 - val_loss: 5.1919 - val_acc: 0.4709
Epoch 665/1000
 - 1s - loss: 0.0102 - acc: 0.9973 - val_loss: 5.2396 - val_acc: 0.4746
Epoch 666/1000
 - 1s - loss: 0.0051 - acc: 0.9993 - val_loss: 5.2962 - val_acc: 0.4520
Epoch 667/1000
 - 1s - loss: 0.0066 - acc: 0.9981 - val_loss: 5.3774 - val_acc: 0.4622
Epoch 668/1000
 - 1s - loss: 0.0052 - acc: 0.9993 - val_loss: 5.4652 - val_acc: 0.4506
Epoch 669/1000
 - 1s - loss: 0.0034

Epoch 751/1000
 - 1s - loss: 0.0042 - acc: 0.9993 - val_loss: 5.8663 - val_acc: 0.4346
Epoch 752/1000
 - 1s - loss: 0.0091 - acc: 0.9981 - val_loss: 5.3989 - val_acc: 0.4593
Epoch 753/1000
 - 1s - loss: 2.8024e-06 - acc: 1.0000 - val_loss: 5.4677 - val_acc: 0.4695
Epoch 754/1000
 - 1s - loss: 0.0062 - acc: 0.9985 - val_loss: 5.6177 - val_acc: 0.4717
Epoch 755/1000
 - 1s - loss: 2.6924e-06 - acc: 1.0000 - val_loss: 5.4656 - val_acc: 0.4709
Epoch 756/1000
 - 1s - loss: 0.0019 - acc: 0.9993 - val_loss: 5.4137 - val_acc: 0.4702
Epoch 757/1000
 - 1s - loss: 0.0033 - acc: 0.9990 - val_loss: 5.3102 - val_acc: 0.4753
Epoch 758/1000
 - 1s - loss: 2.2408e-06 - acc: 1.0000 - val_loss: 5.3914 - val_acc: 0.4804
Epoch 759/1000
 - 1s - loss: 0.0097 - acc: 0.9973 - val_loss: 5.1942 - val_acc: 0.4658
Epoch 760/1000
 - 1s - loss: 0.0080 - acc: 0.9971 - val_loss: 5.3326 - val_acc: 0.4629
Epoch 761/1000
 - 1s - loss: 1.4839e-05 - acc: 1.0000 - val_loss: 5.3566 - val_acc: 0.4658
Epoch 762/1000
 - 1s - loss

Epoch 844/1000
 - 1s - loss: 0.0020 - acc: 0.9990 - val_loss: 5.6421 - val_acc: 0.4469
Epoch 845/1000
 - 1s - loss: 4.4614e-04 - acc: 0.9998 - val_loss: 6.7488 - val_acc: 0.4215
Epoch 846/1000
 - 1s - loss: 0.0042 - acc: 0.9990 - val_loss: 5.5842 - val_acc: 0.4637
Epoch 847/1000
 - 1s - loss: 2.8092e-07 - acc: 1.0000 - val_loss: 5.5237 - val_acc: 0.4731
Epoch 848/1000
 - 1s - loss: 0.0230 - acc: 0.9964 - val_loss: 5.7194 - val_acc: 0.4564
Epoch 849/1000
 - 1s - loss: 0.0072 - acc: 0.9988 - val_loss: 5.4789 - val_acc: 0.4499
Epoch 850/1000
 - 1s - loss: 0.0030 - acc: 0.9990 - val_loss: 5.5888 - val_acc: 0.4528
Epoch 851/1000
 - 1s - loss: 0.0074 - acc: 0.9988 - val_loss: 5.6197 - val_acc: 0.4644
Epoch 852/1000
 - 1s - loss: 4.0015e-07 - acc: 1.0000 - val_loss: 5.6805 - val_acc: 0.4702
Epoch 853/1000
 - 1s - loss: 0.0076 - acc: 0.9990 - val_loss: 5.6008 - val_acc: 0.4586
Epoch 854/1000
 - 1s - loss: 8.4331e-07 - acc: 1.0000 - val_loss: 5.5807 - val_acc: 0.4717
Epoch 855/1000
 - 1s - loss

Epoch 937/1000
 - 1s - loss: 7.5105e-05 - acc: 1.0000 - val_loss: 5.5848 - val_acc: 0.4680
Epoch 938/1000
 - 1s - loss: 1.7887e-07 - acc: 1.0000 - val_loss: 5.5957 - val_acc: 0.4731
Epoch 939/1000
 - 1s - loss: 0.0052 - acc: 0.9985 - val_loss: 5.5591 - val_acc: 0.4753
Epoch 940/1000
 - 1s - loss: 2.7631e-07 - acc: 1.0000 - val_loss: 5.6804 - val_acc: 0.4767
Epoch 941/1000
 - 1s - loss: 0.0037 - acc: 0.9993 - val_loss: 5.7376 - val_acc: 0.4811
Epoch 942/1000
 - 1s - loss: 0.0125 - acc: 0.9985 - val_loss: 5.7482 - val_acc: 0.4600
Epoch 943/1000
 - 1s - loss: 1.3829e-06 - acc: 1.0000 - val_loss: 5.5946 - val_acc: 0.4702
Epoch 944/1000
 - 1s - loss: 0.0065 - acc: 0.9985 - val_loss: 5.5627 - val_acc: 0.4644
Epoch 945/1000
 - 1s - loss: 0.0059 - acc: 0.9990 - val_loss: 5.6058 - val_acc: 0.4622
Epoch 946/1000
 - 1s - loss: 0.0115 - acc: 0.9978 - val_loss: 5.5537 - val_acc: 0.4549
Epoch 947/1000
 - 1s - loss: 2.9862e-04 - acc: 1.0000 - val_loss: 5.4665 - val_acc: 0.4869
Epoch 948/1000
 - 1s - 

In [ ]:
model.save('ECG1.h5')